In [ ]:
import numpy as np
import pandas as pd
import csv

import os
from tqdm.notebook import tqdm

from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

from gsea_api.molecular_signatures_db import GeneSets

In [ ]:
meth_dir = "your/path/here"

In [ ]:
cancer_to_keep = os.listdir(meth_dir)

In [ ]:
cancer_to_keep = np.unique([cancer_to_keep[i].split('_')[0] for i in range(len(cancer_to_keep))]) 

In [ ]:
CIMP_cancers = ['KIRP','SARC','ACC','LAML','MESO','STAD','COAD','LIHC','READ','PCPG','SKCM','THCA','LUAD',
                'LUSC','CESC','KIRC','LGG','HNSC','GBM']

In [ ]:
kegg_file = 'your/path/here'
GOBP_file = 'your/path/here'

kegg_pathways = GeneSets.from_gmt(kegg_file)
GOBP_pathways = GeneSets.from_gmt(GOBP_file)

In [ ]:
kegg_df = kegg_pathways.to_frame()
GOBP_df = GOBP_pathways.to_frame()

In [ ]:
diff_gex_offgene_dir = "your/path/here"
diff_gex_pc = {}
for cancer in sorted(CIMP_cancers):
    diff_gex_pc[cancer] = pd.read_csv(os.path.join(diff_gex_offgene_dir,cancer+"_downstream_genes_gex.csv"))
    

In [ ]:
def get_EASE_contingency(differential_gex,db,pathway,):
    path_db = db.loc[pathway]
    npt = path_db.sum()
    nl = len(differential_gex)
#     nt = 60489 # number of GENCODE ids in the gene counts in TCGA
    nt = 30000 # number of genes in human genome
    npl = len(np.intersect1d(differential_gex,list(path_db[path_db].index)))
    return np.array([[npl-1,npt-npl+1],[nl-npl,nt-nl-(npt-npl)]]),npl,nl

In [ ]:
def get_EASE_pvalue(contingency):
    if contingency[0,0]<1:
        p = 1
    else:
        _,p = fisher_exact(contingency)
    return p

In [ ]:
def compute_enrichment_pvalues(differential_gex,db):
    enrichment_p = pd.DataFrame(columns=["p"])
    if len(differential_gex)==0:
        return enrichment_p 
    
    for pathway in tqdm(db.index):
        contingency,npl,nl = get_EASE_contingency(differential_gex,db,pathway)
        p = get_EASE_pvalue(contingency)
        prop_overlap = npl/nl*100
        enrichment_p = pd.concat([enrichment_p,pd.DataFrame(np.array([p,prop_overlap]).reshape(1,-1),
                                                            index=[pathway],columns=["p","proportion overlap"])])
    return enrichment_p

In [ ]:
def get_enrichment_pc(diff_gex_pc,db):
    enrichment = {}
    for cancer in diff_gex_pc:
        enrichment[cancer] = compute_enrichment_pvalues(diff_gex_pc[cancer].values.ravel(),db)
    return enrichment

In [ ]:
def get_significant_pathways(all_enrichment):
    corrected_ps = multipletests(all_enrichment.p.ravel())[1]
    return pd.DataFrame(corrected_ps,index=all_enrichment.index,columns=["q"])

In [ ]:
kegg_enrich = get_enrichment_pc(diff_gex_pc,kegg_df)

In [ ]:
GOBP_enrich = get_enrichment_pc(diff_gex_pc,GOBP_df)

In [ ]:
significant_ps_pc = {}
for cancer in sorted(CIMP_cancers):
    all_ps = pd.concat([kegg_enrich[cancer],GOBP_enrich[cancer]])
    if all_ps.shape[0]==0:
        significant_ps_pc[cancer] = []
    else:
        significant_ps_pc[cancer] = get_significant_pathways(all_ps)

In [ ]:
downstream_gene_path_dir = "your/path/here"
for cancer in sorted(CIMP_cancers):
    if len(significant_ps_pc[cancer])==0:
        pd.DataFrame(columns=["qs"]).to_csv(os.path.join(downstream_gene_path_dir,cancer+"_downstream_gene_path.csv"))
    else:
        (significant_ps_pc[cancer][significant_ps_pc[cancer].q<0.05]).to_csv(os.path.join(downstream_gene_path_dir,cancer+"_downstream_gene_path.csv"))